<a href="https://colab.research.google.com/github/Dichtael/TDS2324-TrafficAccidents/blob/EDA/EDA_CrashPredictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Predicting Car Crashes in a Given Satellite Image

# Identification of the Research Question

In this data analysis we want to build a car crash predictor, that predicts hotspots of car crashes in a satellite immage by outputting the same satellite image with heatmap like markings.

# Data Collection

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
data = pd.read_csv('/content/all_16_22.csv')
data.head()

,Unnamed: 0,FID;OBJECTID;ULAND;UREGBEZ;UKREIS;UGEMEINDE;UJAHR;UMONAT;USTUNDE;UWOCHENTAG;UKATEGORIE;UART;UTYP1;ULICHTVERH;STRZUSTAND;IstRad;IstPKW;IstFuss;IstKrad;IstGkfz;IstSonstige;LINREFX;LINREFY;XGCSWGS84;YGCSWGS84,OBJECTID;UIDENTSTLA;ULAND;UREGBEZ;UKREIS;UGEMEINDE;UJAHR;UMONAT;USTUNDE;UWOCHENTAG;UKATEGORIE;UART;UTYP1;IstRad;IstPKW;IstFuss;IstKrad;IstSonstige;ULICHTVERH;STRZUSTAND;LINREFX;LINREFY;XGCSWGS84;YGCSWGS84,OBJECTID_1;ULAND;UREGBEZ;UKREIS;UGEMEINDE;UJAHR;UMONAT;USTUNDE;UWOCHENTAG;UKATEGORIE;UART;UTYP1;ULICHTVERH;IstRad;IstPKW;IstFuss;IstKrad;IstGkfz;IstSonstige;STRZUSTAND;LINREFX;LINREFY;XGCSWGS84;YGCSWGS84,OBJECTID;ULAND;UREGBEZ;UKREIS;UGEMEINDE;UJAHR;UMONAT;USTUNDE;UWOCHENTAG;UKATEGORIE;UART;UTYP1;ULICHTVERH;IstRad;IstPKW;IstFuss;IstKrad;IstGkfz;IstSonstige;LINREFX;LINREFY;XGCSWGS84;YGCSWGS84;STRZUSTAND,OBJECTID;UIDENTSTLAE;ULAND;UREGBEZ;UKREIS;UGEMEINDE;UJAHR;UMONAT;USTUNDE;UWOCHENTAG;UKATEGORIE;UART;UTYP1;ULICHTVERH;IstRad;IstPKW;IstFuss;IstKrad;IstGkfz;IstSonstige;LINREFX;LINREFY;XGCSWGS84;YGCSWGS84;STRZUSTAND,OBJECTID;UIDENTSTLAE;ULAND;UREGBEZ;UKREIS;UGEMEINDE;UJAHR;UMONAT;USTUNDE;UWOCHENTAG;UKATEGORIE;UART;UTYP1;ULICHTVERH;STRZUSTAND;IstRad;IstPKW;IstFuss;IstKrad;IstGkfz;IstSonstige;LINREFX;LINREFY;XGCSWGS84;YGCSWGS84
0,0,7.296149e+14,NaN,NaN,NaN,NaN,NaN
1,1,2.454526e+14,NaN,NaN,NaN,NaN,NaN
2,2,8.204025e+14,NaN,NaN,NaN,NaN,NaN
3,3,8.512431e+14,NaN,NaN,NaN,NaN,NaN
4,4,2.194586e+14,NaN,NaN,NaN,NaN,NaN


In [ ]:
data = data.drop(columns=["id"])
data = data.dropna(subset=['UJAHR'])
data['column_name'].fillna(0, inplace=True)
data.head()

KeyError: "['id'] not found in axis"

# Exploratory Data Analysis

In [ ]:
print(data.isnull().sum())

In [ ]:
print(data.info)

**Some Summary Statustics to Explore the Scope of the Data**

In [ ]:
print("In 2013-2017 there were {:,} accidents in ...  .".format(len(data)))
print("{:,} of the accidents ended fatally while in {:,} of the cases the passengers survived.".format(USchwere['fatal'].sum(), USchwere['injured'].sum()+USchwere['minor injury'].sum()))
print("In {:,} of the crashes the light conditions are poor.".format(data['ULICHTVERH'].sum()))

# Geographic Analysis

**Heatmap of Car Crashes in the viewed Region**

In [ ]:
longitude = data['ygcswgs84']
latitude = data['xgcswgs84']

# Erstellung der Heatmap
plt.figure(figsize=(10, 6))
sns.kdeplot(data[longitude], data[latitude], cmap="Reds", shade=True, bw_adjust=0.5)

plt.title('Heatmap von Verkehrsunfällen')
plt.xlabel('Längengrad')
plt.ylabel('Breitengrad')
plt.show()

**Distrubution of Crashes in the Viewed Region**

In [ ]:
plt.figure(figsize=(8,10))
plt.scatter(longitude, latitude, s=0.2, alpha=0.2, color='black')
plt.title('Traffic accidents in Germany from 2016-2020', fontsize=14)
plt.axis('off')
plt.show()

# Temporal Analysis

**Histogram of Crashes over Years and Months**

In [ ]:
# Group by year and month, and calculate the sum of crash counts
monthly_crashes = data.groupby(['jahr', 'monat'])['art'].count().reset_index()
monthly_crashes['crashes'] = monthly_crashes['art']
monthly_crashes = monthly_crashes.drop(columns=['art'])
monthly_crashes.head()

In [ ]:
monthly_crashes.crashes.sum()

In [ ]:
# Create the histogram
plt.figure(figsize=(12, 6))
bars = plt.bar(
    range(len(monthly_crashes)),
    monthly_crashes['crashes'],
    tick_label=monthly_crashes.apply(lambda x: f'{x["jahr"]}-{x["monat"]:02d}', axis=1)
)

# Mark January bars in orange
for i, month in enumerate(monthly_crashes['monat']):
    if month == 1:  # January
        bars[i].set_color('orange')

plt.xlabel('Year-Month')
plt.ylabel('Crash Count')
plt.title('Histogram of Car Crashes by Year-Month')
plt.xticks(rotation=90) # Rotate x-axis labels for better visibility
plt.tight_layout()
plt.show()



*   There appear to be peaks in the occurance of crashes in the months of june and may of every year except for 2020, where the highest amount of car crashes occured in sepember
*   In general the occurance of car crashes seems to rise throughout the displayed time period
*   It becomes more and more logarithmic with tendencies of skewness
*   Further, the traffic reduction resulting of pandemic indused quarantine and homeoffice policies can be observed in the plot between the end of 2020 and start 2021



**Distribution of Crashes throughout the Week**

In [ ]:
# Group by day of week, and calculate the sum of crash counts
weekly_crashes = data.groupby(['wochentag'])['kategorie'].count().reset_index()
weekly_crashes['crashes'] = weekly_crashes['kategorie']
weekly_crashes = weekly_crashes.drop(columns=['kategorie'])
weekly_crashes.head()

In [ ]:
# Create the histogram
plt.figure(figsize=(12, 6))
bars = plt.bar(
    range(len(weekly_crashes)),
    weekly_crashes['crashes'],
    tick_label=weekly_crashes['wochentag']
)

plt.xlabel('Days of the Week')
plt.ylabel('Crash Count')
plt.title('Histogram of Car Crashes throughout the Week')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

**Distribution over the Course of the Hours of a Day**

In [ ]:
# Group by time of day, and calculate the sum of crash counts
daily_crashes = data.groupby(['stunde'])['kategorie'].count().reset_index()
daily_crashes['crashes'] = daily_crashes['kategorie']
daily_crashes = daily_crashes.drop(columns=['kategorie'])
daily_crashes.head()

In [ ]:
# Create the histogram
plt.figure(figsize=(12, 6))
bars = plt.bar(
    range(len(daily_crashes)),
    daily_crashes['crashes'],
    tick_label=daily_crashes['stunde']
)

plt.xlabel('Hour of the day')
plt.ylabel('Crash Count')
plt.title('Histogram of Car Crashes throughout the Day')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Type and nature of Accidents

In [ ]:
# Group by time of day, and calculate the sum of crash counts
typ_crashes = data.groupby(['typ'])['kategorie'].count().reset_index()
typ_crashes['crashes'] = typ_crashes['kategorie']
typ_crashes = typ_crashes.drop(columns=['kategorie'])
data['typ'] = data['typ'].replace(1, 'Fahrunfall')
data['typ'] = data['typ'].replace(2, 'Abbiegeunfall')
data['typ'] = data['typ'].replace(3, 'Kreuzen-Unfall')
data['typ'] = data['typ'].replace(4, 'Überschreiten-Unfall')
data['typ'] = data['typ'].replace(5, 'Unfall durch ruhenden Verkehr')
data['typ'] = data['typ'].replace(6, 'Unfall im Längsverkehr')
data['typ'] = data['typ'].replace(7, 'sonstiger Unfall')
typ_crashes.head()

In [ ]:
# Create the histogram
plt.figure(figsize=(12, 6))
bars = plt.bar(
    range(len(monthly_crashes)),
    typ_crashes['crashes'],
    tick_label=typ_crashes['typ']
)

plt.xlabel('Hour of the day')
plt.ylabel('Crash Count')
plt.title('Histogram of Car Crashes throughout the Day')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

**Relationship between Nature of the Accident and Vehicle Type**

In [ ]:
# Group by time of day, and calculate the sum of crash counts

typ_crashes = data['typ', 'ist_rad', 'ist_pkw', 'ist_fuss', 'ist_gkfz', 'ist_sonstige', 'ist_krad']
typ_crashes['crashes'] = typ_crashes['kategorie']
typ_crashes = typ_crashes.drop(columns=['kategorie'])
data['typ'] = data['typ'].replace(1, 'Fahrunfall')
data['typ'] = data['typ'].replace(2, 'Abbiegeunfall')
data['typ'] = data['typ'].replace(3, 'Kreuzen-Unfall')
data['typ'] = data['typ'].replace(4, 'Überschreiten-Unfall')
data['typ'] = data['typ'].replace(5, 'Unfall durch ruhenden Verkehr')
data['typ'] = data['typ'].replace(6, 'Unfall im Längsverkehr')
data['typ'] = data['typ'].replace(7, 'sonstiger Unfall')
typ_crashes.head()

In [ ]:
sns.relplot(data=data, x="typ", y="tip")

**Distribution of Numerical Values**

In [ ]:
data['kategorie'] = data['kategorie'].replace(0, 'daylight')
data['kategorie'] = data['kategorie'].replace(1, 'dawn')
data['kategorie'] = data['kategorie'].replace(2, 'darkness')

In [ ]:
data['kategorie'] = data['kategorie'].replace(1, 'fatal')
data['kategorie'] = data['kategorie'].replace(2, 'severe')
data['kategorie'] = data['kategorie'].replace(3, 'minor')

In [ ]:
data.hist(figsize=(16, 20), bins=50, xlabelsize=8, ylabelsize=8);



*   ULAND 9, UREGBEZ 2 und UKREIS 61(?) ist am häufigsten Unfallort
* UKATEGORIE 3, UART 5 und UTYP1 6 sind am häufigsten
*   Feiertagsverkehr (14-17) verursacht die meisten Unfälle
*   2019 und Juni sind das Jahr und Monat mit den meisten Unfällen



**Percentage of each Object of Transportation in Realtion to the Absolute Number of Car Crashes**

In [ ]:
isBike = data.ist_rad.sum()
isPkw = data.ist_pkw.sum()
isPedestrian = data.ist_fuss.sum()
isGkfz = data.ist_gkfz.sum()
isKbike = data.ist_krad.sum()
isElse = data.ist_sonstige.sum()

sum = isBike + isPkw + isPedestrian + isGkfz + isKbike + isElse
labels = ['Bike', 'Car', 'Person', 'Truck', 'Motorbike', 'Others']
sizes = [isBike/sum, isPkw/sum, isPedestrian/sum, isGkfz/sum, isKbike/sum, isElse/sum]

fig, ax = plt.subplots()
ax.pie(sizes, labels=labels)



*   Cars are involved in more than half of the total crashes
*   Bikes are involved in a little less than a fourth of the cases



# Data Analysis